In [6]:
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('ner.csv', encoding='unicode_escape', on_bad_lines='skip')

In [8]:
data.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [9]:
df = data[['word','pos','prev-pos']]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050795 entries, 0 to 1050794
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   word      1050783 non-null  object
 1   pos       1050794 non-null  object
 2   prev-pos  1050794 non-null  object
dtypes: object(3)
memory usage: 24.1+ MB


In [11]:
df[df.columns] = df[df.columns].astype('category')

C:\Users\Anantha\AppData\Local\Temp\ipykernel_25640\1119263473.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.columns] = df[df.columns].astype('category')


In [12]:
df = df.fillna(method='ffill')

C:\Users\Anantha\AppData\Local\Temp\ipykernel_25640\569672361.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [13]:
grpW = df.groupby('word', observed=False)
grpPos = df.groupby('pos', observed=False)
grpWPos = df.groupby(['word','pos'], observed=False)
grpPrevPos = df.groupby(['pos', 'prev-pos'], observed=False)

In [16]:
def word_pos(word,grpW):
    return grpW.get_group(word)['pos'].unique()
def argmax(x):
    return x.index(max(x))

def HMM(df, grpW, grpPos, grpWPos, grpPrevPos):
    words = []
    transi_score = []
    back_trace = []
    for i in range(1):
        c = 0
        for j in range(24):
            word = df.iloc[j]['word']
            words.append(word)
            pos = df.iloc[j]['pos']
            prepos = df.iloc[j]['prev-pos']
            if(word=='.'):
                words.remove(word)
                continue
            else:
                ts = []
                bt = []
                for k in word_pos(word,grpW):
                    emmision_prob = len(grpWPos.get_group((word,k))) / len(grpPos.get_group(k))
                    for i in word_pos(words[c-1], grpW):
                        trans_prob = len(grpPrevPos.get_group((k,i))) / len(grpPos.get_group(i))
                        ts.append(emmision_prob*trans_prob)
                        bt.append(i)

                transi_score.append(ts)
                back_trace.append(bt)
                c=c+1
    posTagging = []

    for ele in range(len(transi_score)):
        posTagging.append(back_trace[ele][argmax(transi_score[ele])])
    print(back_trace)
    print(posTagging[1:])
    print(words)
                

In [17]:
HMM(df,grpW,grpPos,grpWPos,grpPrevPos)

[['NNS'], ['NNS', 'NNS'], ['IN', 'RP'], ['NNS', 'NNS'], ['VBP', 'VB', 'VBP', 'VB'], ['VBN', 'VBD'], ['IN'], ['NNP', 'NNP', 'NNP'], ['TO', 'IN', 'CC', 'TO', 'IN', 'CC'], ['VB', 'NN', 'VB', 'NN'], ['DT', 'NNP'], ['NN', 'NN'], ['IN', 'RP'], ['NNP'], ['CC', 'CC'], ['VB', 'NN', 'VB', 'NN'], ['DT', 'NNP'], ['NN', 'NN'], ['IN', 'RP', 'IN', 'RP', 'IN', 'RP', 'IN', 'RP'], ['JJ', 'NNP', 'NNS', 'NN'], ['NNS'], ['IN', 'IN', 'IN', 'IN'], ['DT', 'IN', 'WDT', 'RB']]
['NNS', 'RP', 'NNS', 'VBP', 'VBN', 'IN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'CC', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT']
['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country']
